In [1]:
!nvidia-smi

Thu Jan 12 18:55:58 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.106.00   Driver Version: 460.106.00   CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-DGXS...  Off  | 00000000:07:00.0 Off |                    0 |
| N/A   45C    P0    54W / 300W |  29954MiB / 32508MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  Tesla V100-DGXS...  Off  | 00000000:08:00.0 Off |                    0 |
| N/A   

In [2]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="3"

In [3]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

2023-01-12 18:56:10.005210: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-12 18:56:10.301316: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 15649749367647460464
xla_global_id: -1
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 32476168192
locality {
  bus_id: 1
  links {
  }
}
incarnation: 10653532408418105748
physical_device_desc: "device: 0, name: Tesla V100-DGXS-32GB, pci bus id: 0000:0f:00.0, compute capability: 7.0"
xla_global_id: 416903419
]


2023-01-12 18:56:14.424589: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-12 18:56:15.282216: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1616] Created device /device:GPU:0 with 30971 MB memory:  -> device: 0, name: Tesla V100-DGXS-32GB, pci bus id: 0000:0f:00.0, compute capability: 7.0


In [4]:
import gc
import gdown
import numpy as np
import pandas  as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
from collections import defaultdict
import tqdm.notebook as tqdm

In [5]:
aspect = ['biological','process']
aspect_abbr = 'bp'

In [6]:
file = '_'.join(aspect)+'.csv'
if not os.path.exists(file):
    url = "https://drive.google.com/file/d/1RyeLQPFTMWAIr-OzELTWIx60ln-mZ7g_/view?usp=sharing"
    output = file
    gdown.download(url=url, output=output, quiet=False, fuzzy=True)

Downloading...
From: https://drive.google.com/uc?id=1RyeLQPFTMWAIr-OzELTWIx60ln-mZ7g_
To: /home/191it109/project/home/biological_process.csv
100%|██████████████████████████████████████| 34.6M/34.6M [00:00<00:00, 39.7MB/s]


In [7]:
df = pd.read_csv(file)
df

,Entry,Entry Name,Organism,Length,Sequence,Gene Ontology (biological process)
0,F4KFS5,GMI1_ARATH,Arabidopsis thaliana (Mouse-ear cress),1598,MSSRRSVKRSLVLDDDDDEDIFYNFKVLLPNGTSVKLTLKNPEPEI...,GO:0000724
1,B4ESY8,URK_PROMH,Proteus mirabilis (strain HI4320),213,MADTAHQCTIVGIAGASASGKSLIASTLYRELRAQVGDHNIGVIPE...,GO:0016310
2,Q9H479,FN3K_HUMAN,Homo sapiens (Human),309,MEQLLRAELRTATLRAFGGPGAGCISEGRAYDTDAGPVFVKVNRRT...,GO:0016310
3,B4PRE2,DGKH_DROYA,Drosophila yakuba (Fruit fly),1917,MSHLKLDTLHVQRSPRGSRRSSRSSGRSSACSSGSISPVPIIPIIS...,GO:0016310
4,P17129,PSPB_CANLF,Canis lupus familiaris (Dog) (Canis familiaris),363,LLWLLLLPTLCGLGAADWSAPSLACARGPAFWCQSLEQALQCRALG...,GO:0006629
...,...,...,...,...,...,...
72465,P44371,RS2_HAEIN,Haemophilus influenzae (strain ATCC 51907 / DS...,240,MAQVSMRDMINAGVHFGHQTRYWNPQMKPFIFGARNGVHIINLEKT...,GO:0006412
72466,A1V880,RS13_BURMS,Burkholderia mallei (strain SAVP1),121,MARIAGVNIPNHQHTEIGLTAIFGIGRTRARSICVASGVAFSKKVK...,GO:0006412
72467,Q04955,FLIG_CAUVC,Caulobacter vibrioides (strain ATCC 19089 / CB...,340,MAMKLAVNDVKNLSGPEKAAIVLLALGEDHTRIWEALDDEEIKEVS...,GO:0006935
72468,B5R0P3,PURA_SALEP,Salmonella enteritidis PT4 (strain P125109),432,MGNNVVVLGTQWGDEGKGKIVDLLTERAKYVVRYQGGHNAGHTLVI...,GO:0044208


In [8]:
go_terms_bp = set()
for idx, row in df.iterrows():
    for term in row['Gene Ontology ('+' '.join(aspect)+')'].split(';'):
        go_terms_bp.add(term)
go_terms_bp = list(go_terms_bp)
go_terms_bp.sort()
print(len(go_terms_bp))
print(go_terms_bp)

201
['GO:0000027', 'GO:0000103', 'GO:0000105', 'GO:0000122', 'GO:0000162', 'GO:0000209', 'GO:0000398', 'GO:0000724', 'GO:0000902', 'GO:0000917', 'GO:0001525', 'GO:0001934', 'GO:0002098', 'GO:0002181', 'GO:0002250', 'GO:0002949', 'GO:0005975', 'GO:0005978', 'GO:0006006', 'GO:0006094', 'GO:0006096', 'GO:0006099', 'GO:0006164', 'GO:0006166', 'GO:0006189', 'GO:0006207', 'GO:0006226', 'GO:0006235', 'GO:0006241', 'GO:0006260', 'GO:0006275', 'GO:0006281', 'GO:0006284', 'GO:0006289', 'GO:0006298', 'GO:0006310', 'GO:0006325', 'GO:0006338', 'GO:0006351', 'GO:0006353', 'GO:0006355', 'GO:0006357', 'GO:0006364', 'GO:0006366', 'GO:0006396', 'GO:0006397', 'GO:0006400', 'GO:0006401', 'GO:0006402', 'GO:0006412', 'GO:0006417', 'GO:0006420', 'GO:0006424', 'GO:0006426', 'GO:0006432', 'GO:0006433', 'GO:0006457', 'GO:0006468', 'GO:0006470', 'GO:0006508', 'GO:0006511', 'GO:0006520', 'GO:0006526', 'GO:0006541', 'GO:0006556', 'GO:0006605', 'GO:0006629', 'GO:0006633', 'GO:0006730', 'GO:0006744', 'GO:0006777', '

In [9]:
def get_segments(sequence,segment_size=100,gap=30):
    segments = []
    start = 0
    end = segment_size
    while end <= len(sequence):
        segments.append(sequence[start:end])
        start += gap
        end += gap
    last_segment = sequence[start:]
    segments.append(last_segment)
    
    return segments

def get_training_data(df,segment_size=100,gap=30):
    training_data = list()
    for idx,row in tqdm.tqdm(df.iterrows()):
        labels = [0] * len(go_terms_bp)
        for term in row['Gene Ontology ('+' '.join(aspect)+')'].split(';'):
            labels[go_terms_bp.index(term)] = 1
        segments = get_segments(row['Sequence'],segment_size,gap)
        for segment in segments:
            training_data.append([row['Entry'],segment,labels])
    return training_data

In [10]:
training_data = get_training_data(df)
print(len(training_data))

0it [00:00, ?it/s]

794486


In [28]:
def get_ngrams(segment,n=3):
    ngrams = []
    for i in range(len(segment)-n+1):
        ngrams.append(segment[i:i+n])
    return ngrams

In [29]:
# Generate training data of ngrams
if os.path.exists('bp/training_data_3grams.npy'):
    print('Loading saved ngrams...')
    training_data_ngrams = np.load('bp/training_data_3grams.npy',allow_pickle=True)
else:
    print('Preparing from scratch...')
    training_data_ngrams = []

    for i in tqdm.tqdm(range(len(training_data))):
        training_data_ngrams.append([training_data[i][0],get_ngrams(training_data[i][1],n=3),training_data[i][2]])
        
    np.save('bp/training_data_3grams.npy',training_data_ngrams)
    
print(len(training_data_ngrams))

Loading saved ngrams...
794486


In [30]:
def get_skip_grams(segment,skip=1,n=3):
    skip_grams = []
    window_size = skip + n
    for i in range(len(segment)-window_size+1):
        window = segment[i:i+window_size]
        indices = list(range(window_size))
        indices.pop(0)
        for idx in indices[::-1]:
            temp = ''
            for j in range(window_size):
                if j!=idx:
                    temp+=window[j]
            skip_grams.append(temp)

    return skip_grams

In [31]:
if os.path.exists('bp/training_data_skip1_3grams.npy'):
    print('Loading saved skip grams...')
    training_data_skip_grams = np.load('bp/training_data_skip1_3grams.npy',allow_pickle=True)
else:
    print('Preparing from scratch...')
    training_data_skip_grams = []
    for i in tqdm.tqdm(range(len(training_data))):
        training_data_skip_grams.append([training_data[i][0],get_skip_grams(training_data[i][1],n=3),training_data[i][2]])
    np.save('bp/training_data_skip1_3grams.npy',training_data_skip_grams)
print(len(training_data_skip_grams))

Loading saved skip grams...
794486


In [13]:
import tensorflow as tf
import tensorflow_addons as tfa

In [14]:
#Assuming training_data as global variable

def train_test_split(X,y,fold_no,prev_index,Kfolds=5):
    test_split = 1/Kfolds
    
    start_index = prev_index
    end_index = (fold_no + 1) * (test_split) * len(X)
    end_index = round(end_index)
    
    if end_index==len(X):
        end_index -= 1
    
    entry = training_data[end_index][0]
    entries = [sample[0] for sample in training_data]
    
    first_occurence = entries.index(entry)
    entries.reverse()
    
    last_occurence = entries.index(entry)
    last_occurence = len(entries) - last_occurence - 1
    
    del entries
    gc.collect()
    
    end_index = first_occurence if (abs(end_index-first_occurence) < abs(end_index-last_occurence)) else last_occurence
    
    X_test = X[start_index:end_index+1]
    y_test = y[start_index:end_index+1]
    X_train = X[:start_index]
    X_train.extend(X[end_index+1:])
    y_train = y[:start_index]
    y_train.extend(y[end_index+1:])
    
    return X_train, y_train, X_test, y_test, start_index, end_index + 1

In [15]:
MAX_WORDS = 13824
MAX_LEN_NG = 98 #100
MAX_LEN_SG = 291 #300

def tokenization(X_train,X_test,maxlen):

    tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=MAX_WORDS)
    tokenizer.fit_on_texts(X_train)

    X_train = tokenizer.texts_to_sequences(X_train)
    X_test = tokenizer.texts_to_sequences(X_test)

    vocab_size = len(tokenizer.word_index) + 1

    X_train = tf.keras.preprocessing.sequence.pad_sequences(X_train, padding='post', maxlen=maxlen)
    X_test = tf.keras.preprocessing.sequence.pad_sequences(X_test, padding='post', maxlen=maxlen)
    
    return X_train, X_test, vocab_size, tokenizer

In [16]:
from tensorflow.keras import backend as K

false_negative_penalty = 6
false_positive_penalty = 1

def custom_loss(y_true, y_logit):

    loss = float(0)
    y_true = tf.cast(y_true, tf.float32)
    y_logit = tf.cast(y_logit, tf.float32)
    
    first_term = false_negative_penalty * float(y_true) * - K.log(y_logit + K.epsilon())
    second_term = false_positive_penalty * (1 - float(y_true)) * - K.log(1 - y_logit + K.epsilon())
    
    loss = K.mean(first_term+second_term)

    return loss

def precision(y_true, y_pred):
    
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)), axis=1)
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)), axis=1)
    precision = true_positives / (predicted_positives + K.epsilon())
    
    return K.mean(precision)


def recall(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)), axis=1)
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)), axis=1)
    recall = true_positives / (possible_positives + K.epsilon())
    return K.mean(recall)

def f1_score(y_true, y_pred):
    rec = recall(y_true,y_pred)
    prec = precision(y_true,y_pred)
    f1 = 2*prec*rec/(prec+rec)
    return f1

In [17]:
class attention(tf.keras.layers.Layer):
    
    def __init__(self, return_sequences=True,**kwargs):
        self.return_sequences = return_sequences
        super(attention,self).__init__()

    def get_config(self):
      config = super().get_config().copy()
      config.update({
          'return_sequences': self.return_sequences 
      })
      return config
        
    def build(self, input_shape):
        
        self.W=self.add_weight(name="att_weight", shape=(input_shape[-1],1),
                               initializer="normal")
        self.b=self.add_weight(name="att_bias", shape=(input_shape[1],1),
                               initializer="zeros")
        
        super(attention,self).build(input_shape)
        
    def call(self, x):
        
        e = K.tanh(K.dot(x,self.W)+self.b)
        a = K.softmax(e, axis=1)
        output = x*a
        
        if self.return_sequences:
            return output
        
        return K.sum(output, axis=1)

In [18]:
#Base model using ngrams 
NUM_CLASSES = 201

def get_model_ng(vocab_size_ng):
    input_ngrams = tf.keras.layers.Input(shape=(MAX_LEN_NG,))

    embedding_layer = tf.keras.layers.Embedding(vocab_size_ng, 32)(input_ngrams)

    LSTM_Layer_1 = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(70,return_sequences=True))(embedding_layer)
    attention_output_1 = attention(return_sequences=False)(LSTM_Layer_1)
    dropout = tf.keras.layers.Dropout(0.3)(attention_output_1)
    dense_layer_1 = tf.keras.layers.Dense(NUM_CLASSES, activation='sigmoid')(dropout)

    model = tf.keras.models.Model(
        inputs=input_ngrams, 
        outputs=dense_layer_1
    )
    
    return model

In [19]:
#Base model using skip grams 
NUM_CLASSES = 201

def get_model_sg(vocab_size_sg):
    input_skip_grams = tf.keras.layers.Input(shape=(MAX_LEN_SG,))

    embedding_layer = tf.keras.layers.Embedding(vocab_size_sg, 32)(input_skip_grams)

    LSTM_Layer_1 = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(70,return_sequences=True))(embedding_layer)
    attention_output_1 = attention(return_sequences=False)(LSTM_Layer_1)
    dropout = tf.keras.layers.Dropout(0.3)(attention_output_1)
    dense_layer_1 = tf.keras.layers.Dense(NUM_CLASSES, activation='sigmoid')(dropout)

    model = tf.keras.models.Model(
        inputs=input_skip_grams, 
        outputs=dense_layer_1
    )
    
    return model

In [20]:
NUM_CLASSES = 201 #For bp (Change according to aspects)

def get_model_ng_sg(vocab_size_ng, vocab_size_sg):
    #Input layers

    input_ngrams = tf.keras.layers.Input(shape=(MAX_LEN_NG,)) 
    input_skip_grams = tf.keras.layers.Input(shape=(MAX_LEN_SG,)) 

    #embeddings
    embedding_layer_ngrams = tf.keras.layers.Embedding(vocab_size_ng, 32)(input_ngrams)
    embedding_layer_skip_grams = tf.keras.layers.Embedding(vocab_size_sg, 32)(input_skip_grams)

    #BI-LSTMs for each of the inputs
    sequence_output_1 = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(70,return_sequences=True))(embedding_layer_ngrams)
    attention_output_1 = attention(return_sequences=False)(sequence_output_1)
    dropout_1 = tf.keras.layers.Dropout(0.3)(attention_output_1)
    dense_layer_1 = tf.keras.layers.Dense(NUM_CLASSES, activation='sigmoid')(dropout_1)

    sequence_output_2 = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(70, return_sequences=True))(embedding_layer_skip_grams)
    attention_output_2 = attention(return_sequences=False)(sequence_output_2)
    dropout_2 = tf.keras.layers.Dropout(0.3)(attention_output_2)
    dense_layer_2 = tf.keras.layers.Dense(NUM_CLASSES, activation='sigmoid')(dropout_2)

    max_layer = tf.keras.layers.Maximum()([dense_layer_1,dense_layer_2])

    model = tf.keras.models.Model(
        inputs=[
            input_ngrams,
            input_skip_grams
        ], 
        outputs=max_layer)
    
    return model

In [21]:
def compute_metrics(predictions, start_index, end_index):
    final_predictions = []
    actual_y_test = []

    current_entry = ''
    counter = 0
    total_counts = 0

    if len(predictions) == len(training_data[start_index: end_index]):
        temp = np.zeros(NUM_CLASSES)
        for i in range(len(predictions)):
            if current_entry != training_data[start_index+i][0]:
                #compute prev
                if i!=0:
                    temp /= counter
                    final_predictions.append(temp)

                #reset
                total_counts += counter
                counter = 1
                temp = np.zeros(NUM_CLASSES)

                #init new
                current_entry = training_data[start_index+i][0]
                temp += np.array(predictions[i])
                actual_y_test.append(training_data[start_index+i][2])
            else:
                temp += np.array(predictions[i])
                counter += 1

        total_counts += counter
        temp /= counter
        final_predictions.append(temp)

    else:
        print('Lengths of predictions dont match with test data')
    
    final_predictions = np.array(final_predictions, dtype=float)
    actual_y_test = np.array(actual_y_test, dtype=float)
    
    rec = recall(actual_y_test,final_predictions)
    prec = precision(actual_y_test,final_predictions)
    f1 = f1_score(actual_y_test,final_predictions)
    
    return rec,prec,f1

### Model using ngrams only

In [22]:
#Considering ngrams
X_ng = [' '.join(sample[1]) for sample in training_data_ngrams]
y = [sample[2] for sample in training_data]

del training_data_ngrams

In [23]:
Kfolds = 5
prev_index = 0

recs = []
precs = []
f1s = []

for i in range(Kfolds):
    print('\n\n****Fold:', i+1,'****')
    
    print('Splitting into train-test...')
    X_train_ng, y_train, X_test_ng, y_test, start_index, prev_index = train_test_split(X_ng,y,i,prev_index,Kfolds)

    print('Tokenizing...')
    X_train_ng, X_test_ng, vocab_size_ng, tokenizer1 = tokenization(X_train_ng, X_test_ng, MAX_LEN_NG)
    
    print('Shuffling...')
    shuffled = [[X_train_ng[i],y_train[i]] for i in range(len(X_train_ng))]
    np.random.shuffle(shuffled)

    X_train_ng = [shuffled[i][0] for i in range(len(shuffled))]
    y_train = [shuffled[i][1] for i in range(len(shuffled))]
    X_train_ng = np.array(X_train_ng)
    y_train = np.array(y_train)
    
    
    
    model = get_model_ng(vocab_size_ng)
    
    model.compile(
        loss=custom_loss, 
        optimizer='adam', 
        metrics=[
            recall,
            precision,
            f1_score
        ])
    
    print('Training...')
    history = model.fit(X_train_ng, y_train, batch_size=32, epochs=5,validation_split=0.2)
    
    print('\nEvaluating model...')
    predictions = model.predict(X_test_ng)
    
    rec, prec, f1 = compute_metrics(predictions, start_index, prev_index)
    
    recs.append(rec.numpy())
    precs.append(prec.numpy())
    f1s.append(f1.numpy())

print('Recall:',sum(recs)/len(recs))
print('Precision:',sum(precs)/len(precs))
print('F1-Score:',sum(f1s)/len(f1s))



****Fold: 1 ****
Splitting into train-test...
Tokenizing...
Shuffling...


2023-01-12 19:34:38.409321: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1616] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 30971 MB memory:  -> device: 0, name: Tesla V100-DGXS-32GB, pci bus id: 0000:0f:00.0, compute capability: 7.0


Training...
Epoch 1/5


2023-01-12 19:34:44.786692: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8100


15890/15890 [==============================] - 184s 11ms/step - loss: 0.1645 - recall: 0.0508 - precision: 0.0420 - f1_score: nan - val_loss: 0.1416 - val_recall: 0.1206 - val_precision: 0.1068 - val_f1_score: nan
Epoch 2/5
15890/15890 [==============================] - 178s 11ms/step - loss: 0.1313 - recall: 0.1879 - precision: 0.1583 - f1_score: nan - val_loss: 0.1186 - val_recall: 0.2433 - val_precision: 0.2211 - val_f1_score: 0.2308
Epoch 3/5
15890/15890 [==============================] - 178s 11ms/step - loss: 0.1136 - recall: 0.3020 - precision: 0.2514 - f1_score: 0.2733 - val_loss: 0.1062 - val_recall: 0.3378 - val_precision: 0.2983 - val_f1_score: 0.3160
Epoch 4/5
15890/15890 [==============================] - 178s 11ms/step - loss: 0.1027 - recall: 0.3743 - precision: 0.3143 - f1_score: 0.3407 - val_loss: 0.0989 - val_recall: 0.3961 - val_precision: 0.3526 - val_f1_score: 0.3723
Epoch 5/5
15890/15890 [==============================] - 178s 11ms/step - loss: 0.0952 - recall: 0.

### Model using skip grams only

In [26]:
#Considering skip grams
X_sg = [' '.join(sample[1]) for sample in training_data_skip_grams]
y = [sample[2] for sample in training_data]

del training_data_skip_grams

In [27]:
Kfolds = 5
prev_index = 0

recs = []
precs = []
f1s = []

for i in range(Kfolds):
    print('\n\n****Fold:', i+1,'****')
    
    print('Splitting into train-test...')
    X_train_sg, y_train, X_test_sg, y_test, start_index, prev_index = train_test_split(X_sg,y,i,prev_index,Kfolds)

    print('Tokenizing...')
    X_train_sg, X_test_sg, vocab_size_sg, tokenizer1 = tokenization(X_train_sg, X_test_sg, MAX_LEN_SG)
    
    print('Shuffling...')
    shuffled = [[X_train_sg[i],y_train[i]] for i in range(len(X_train_sg))]
    np.random.shuffle(shuffled)

    X_train_sg = [shuffled[i][0] for i in range(len(shuffled))]
    y_train = [shuffled[i][1] for i in range(len(shuffled))]
    X_train_sg = np.array(X_train_sg)
    y_train = np.array(y_train)
    

    model = get_model_sg(vocab_size_sg)
    
    model.compile(
        loss=custom_loss, 
        optimizer='adam', 
        metrics=[
            recall,
            precision,
            f1_score
        ])
    
    print('Training...')
    history = model.fit(X_train_sg, y_train, batch_size=32, epochs=5,validation_split=0.2)
    
    print('Evaluating model...')
    predictions = model.predict(X_test_sg)
    
    rec, prec, f1 = compute_metrics(predictions, start_index, prev_index)
    
    recs.append(rec.numpy())
    precs.append(prec.numpy())
    f1s.append(f1.numpy())

print('Recall:',sum(recs)/len(recs))
print('Precision:',sum(precs)/len(precs))
print('F1-Score:',sum(f1s)/len(f1s))



****Fold: 1 ****
Splitting into train-test...
Tokenizing...
Shuffling...
Training...
Epoch 1/5
15890/15890 [==============================] - 386s 24ms/step - loss: 0.1637 - recall: 0.0490 - precision: 0.0411 - f1_score: nan - val_loss: 0.1446 - val_recall: 0.1165 - val_precision: 0.0972 - val_f1_score: nan
Epoch 2/5
15890/15890 [==============================] - 381s 24ms/step - loss: 0.1352 - recall: 0.1616 - precision: 0.1362 - f1_score: nan - val_loss: 0.1244 - val_recall: 0.2110 - val_precision: 0.1873 - val_f1_score: 0.1975
Epoch 3/5
15890/15890 [==============================] - 382s 24ms/step - loss: 0.1178 - recall: 0.2703 - precision: 0.2256 - f1_score: nan - val_loss: 0.1106 - val_recall: 0.3076 - val_precision: 0.2708 - val_f1_score: 0.2871
Epoch 4/5
15890/15890 [==============================] - 380s 24ms/step - loss: 0.1065 - recall: 0.3455 - precision: 0.2894 - f1_score: 0.3139 - val_loss: 0.1028 - val_recall: 0.3601 - val_precision: 0.3210 - val_f1_score: 0.3387
Epoch

### Model using skip grams and ngrams

In [32]:
#Considering ngrams and skip grams
X_ng = [' '.join(sample[1]) for sample in training_data_ngrams]
X_sg = [' '.join(sample[1]) for sample in training_data_skip_grams]
y = [sample[2] for sample in training_data]

del training_data_ngrams
del training_data_skip_grams

In [33]:
print(len(X_ng),len(X_sg))

794486 794486


In [34]:
Kfolds = 5
prev_index = 0

recs = []
precs = []
f1s = []

for i in range(Kfolds):
    print('\n\n****Fold:', i+1,'****')
    
    print('Splitting into train-test...')
    X_train_ng, y_train, X_test_ng, y_test, start_index, prev_index1 = train_test_split(X_ng,y,i,prev_index,Kfolds)
    X_train_sg, _, X_test_sg, _, _, _= train_test_split(X_sg,y,i,prev_index,Kfolds)
    
    prev_index = prev_index1
    
    print('Tokenizing...')
    X_train_ng, X_test_ng, vocab_size_ng, tokenizer1 = tokenization(X_train_ng, X_test_ng, MAX_LEN_NG)  
    X_train_sg, X_test_sg, vocab_size_sg, tokenizer2 = tokenization(X_train_sg, X_test_sg, MAX_LEN_SG)
    
   
    print('Shuffling...')   
    shuffled = [[X_train_ng[i],X_train_sg[i],y_train[i]] for i in range(len(X_train_ng))]
    np.random.shuffle(shuffled)

    X_train_ng = [shuffled[i][0] for i in range(len(shuffled))]
    X_train_sg = [shuffled[i][1] for i in range(len(shuffled))]
    y_train = [shuffled[i][2] for i in range(len(shuffled))]
    X_train_ng = np.array(X_train_ng)
    X_train_sg = np.array(X_train_sg)
    y_train = np.array(y_train)
    
    
    model = get_model_ng_sg(vocab_size_ng, vocab_size_sg)
    
    model.compile(
        loss=custom_loss, 
        optimizer='adam', 
        metrics=[
            recall,
            precision,
            f1_score
        ])

    print('Training...')
    history = model.fit([X_train_ng,X_train_sg], y_train, batch_size=32, epochs=5,validation_split=0.2)
    
    print('Evaluating model...')
    predictions = model.predict([X_test_ng,X_test_sg])
    
    rec, prec, f1 = compute_metrics(predictions, start_index, prev_index)
    
    recs.append(rec.numpy())
    precs.append(prec.numpy())
    f1s.append(f1.numpy())

print('Recall:',sum(recs)/len(recs))
print('Precision:',sum(precs)/len(precs))
print('F1-Score:',sum(f1s)/len(f1s))



****Fold: 1 ****
Splitting into train-test...
Tokenizing...
Shuffling...
Training...
Epoch 1/5
15890/15890 [==============================] - 526s 33ms/step - loss: 0.1604 - recall: 0.0736 - precision: 0.0629 - f1_score: nan - val_loss: 0.1374 - val_recall: 0.1604 - val_precision: 0.1328 - val_f1_score: nan
Epoch 2/5
15890/15890 [==============================] - 520s 33ms/step - loss: 0.1259 - recall: 0.2283 - precision: 0.1861 - f1_score: nan - val_loss: 0.1141 - val_recall: 0.2981 - val_precision: 0.2565 - val_f1_score: 0.2748
Epoch 3/5
15890/15890 [==============================] - 520s 33ms/step - loss: 0.1065 - recall: 0.3581 - precision: 0.2964 - f1_score: 0.3232 - val_loss: 0.1003 - val_recall: 0.3948 - val_precision: 0.3503 - val_f1_score: 0.3704
Epoch 4/5
15890/15890 [==============================] - 516s 33ms/step - loss: 0.0942 - recall: 0.4379 - precision: 0.3703 - f1_score: 0.4003 - val_loss: 0.0919 - val_recall: 0.4504 - val_precision: 0.4037 - val_f1_score: 0.4251
Ep